In [1]:
import numpy
import cvxpy as cvx
import control

## Solving the (continuous) Lyapunov equation
$$ A^TX +XA + Q = 0 $$

Find $X>0$ (positive semi-definite) given $P,Q > 0$ and $A$ is Hurwitz (globally asymptotically stable).

In [2]:
A = np.diag([-4, -5,-.01])
# generate a random Hurwitz A matrix
# sys = control.rss(3,1,1)
# A = sys.A
Q = np.diag([1,1,10])

### Solve with python control system library
(which calls `slycot`, which calls the Fortran library `slicot`)

In [3]:
Xcontrol = control.lyap( A, Q )
print(Xcontrol)

[[1.25e-01 0.00e+00 0.00e+00]
 [0.00e+00 1.00e-01 0.00e+00]
 [0.00e+00 0.00e+00 5.00e+02]]


### Solve as an LMI/SDP
Problem already setup as an SDP!

In [4]:
n = A.shape[0]
# construct an nxn positive semi-definite matrix
X = cvx.Variable( (n,n), PSD=True) 
# setup the constraint
constr = [ A.T@X + X@A + Q == np.zeros((n,n))]
# problem is a feasibility problem, objective unimportant
obj = cvx.Minimize( None )
prob = cvx.Problem( obj, constr)
#solve with an SDP solver (in this case SCS)
prob.solve()

if prob.solution.status == 'infeasible':
    raise RuntimeError("Problem is infeasible")

Xsdp = X.value
print(Xsdp)

[[1.25e-01 0.00e+00 0.00e+00]
 [0.00e+00 1.00e-01 0.00e+00]
 [0.00e+00 0.00e+00 5.00e+02]]


Another way to specify a positive semidefinite matrix

In [5]:
X = cvx.Variable( (n,n) )
constr = [ X.T == X, X>> 0] # X is PSD
constr.append( A.T@X + X@A + Q == np.zeros((n,n)) )
obj = cvx.Minimize( None )
prob = cvx.Problem( obj, constr)
prob.solve();

In [6]:
X.value

array([[1.25e-01, 0.00e+00, 0.00e+00],
       [0.00e+00, 1.00e-01, 0.00e+00],
       [0.00e+00, 0.00e+00, 5.00e+02]])

Another method of setting up the feasibility problem is as an optimization problem Dullerud and Paganini 2000 $\S1.4$:

$$
\text{min } t \\
\text{subject to } A^TX + XA +Q -tI \le 0
$$

The original problem is feasible iff $t<0$.

At least as specified, if the problem is already feasible, giving the above formulation to a solver is not bounded.

In [7]:
X = cvx.Variable( (n,n) , PSD=True)
t = cvx.Variable()
I = np.identity(n)
constr = [ A.T@X + X@A +Q -t*I << 0]
obj = cvx.Minimize( t )
prob = cvx.Problem(obj,constr)
prob.solve()
print(t.value)

None


In [8]:
prob.solution.status

'unbounded'

Bounding $t$ returns a solution

In [9]:
constr.append( t >= -100 )
prob = cvx.Problem(obj,constr)
prob.solve()
print(t.value)

-99.9999942531599


In [10]:
prob.solution.status

'optimal'

### SDP approach not always the fastest
In this case, a faster approach comes from linear algebra.
See `slicot`:  https://github.com/KTH-AC/slicot/blob/master/src/SB03MD.f#L182

In [11]:
n = 5 # dimension of problem
np.random.seed(1) 
Q = np.diag( np.random.rand(n) * -1)

In [12]:
#set random seed, both sets of A matrices are the same
# for both tests
np.random.seed(123)

In [13]:
# setup a parameterized optimization problem
# a cvx.Parameter can be changed without reconstructing the problem
# useful in cases where the same optimization problem is used repeadetdly
A = cvx.Parameter(shape=(n,n))
X = cvx.Variable( (n,n), PSD=True) 
# setup the constraint
constr = [ A.T@X + X@A + Q == np.zeros((n,n))]
# problem is a feasibility problem, objective unimportant
obj = cvx.Minimize( None )
prob = cvx.Problem( obj, constr)

`control.rss(n,i,o)` generates a stable random state space system with $n$ states, $i$ inputs, and $o$ outputs.

In [14]:
%%timeit
A.value = control.rss(n,1,1).A
prob.solve();

25.1 ms ± 2.28 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [15]:
# reset random seed
np.random.seed(123)

In [16]:
%%timeit
A = control.rss(n,1,1).A
X = control.lyap( A, Q )

435 µs ± 4.31 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
